# Libraries

In [1]:
import numpy as np
import pandas as pd
import operator
from copy import deepcopy
from math import log

# Load Data

In [2]:
filepath = './data/spliceATrainKIS.dat'
data = []

In [3]:
with open(filepath) as fp:
    line = fp.readline()
    cnt = 1
    while line:
        data.append(line.strip())
        line = fp.readline()
        cnt += 1

In [4]:
np.shape(data)

(11577,)

In [5]:
cutNr = int(data[0])

In [6]:
cutNr

68

In [7]:
data = data[1:]

In [8]:
data = np.array(data)

In [9]:
data = np.reshape(a=data, newshape=(int(11576/2), 2))

In [10]:
df = pd.DataFrame(data, columns=['y', "seq"])

In [11]:
df.y = pd.to_numeric(df.y)
classes = list(df.y)

In [12]:
attributes = set("".join([i for i in df.seq]))

In [13]:
sequences = np.array([list(i) for i in df.seq])

In [14]:
attributes


{'A', 'C', 'G', 'N', 'S', 'T'}

# Algorytm 

zbiór klas 'c'- czy jest prawdziwy czy nie (1 & 0)

zbiór obiektów 's' - zbiór kolejnych przykładów

zbiór atrybutów poza klasą 'r'- zbiór unikalnych liter

## częstość i-tej klasy

In [15]:
#znalezc ceche, ktora ma najwieksza czestosc
p = [{c: 0 for c in attributes} for i in range(sequences.shape[1])]
n = deepcopy(p)
pAn = deepcopy(p)
for ridx, row in enumerate(sequences):
    for cidx, column in enumerate(row):
        pAn[cidx][column] += 1
        if df.y[ridx] == 1:
            p[cidx][column] += 1
        else:
            n[cidx][column] += 1
        

mam zliczoną ilość wystąpień klejnych klas w danych numerach algorytmu

In [16]:
p[:5]

[{'C': 254, 'A': 262, 'G': 308, 'T': 290, 'N': 2, 'S': 0},
 {'C': 266, 'A': 265, 'G': 319, 'T': 263, 'N': 3, 'S': 0},
 {'C': 287, 'A': 255, 'G': 292, 'T': 281, 'N': 1, 'S': 0},
 {'C': 250, 'A': 260, 'G': 337, 'T': 268, 'N': 1, 'S': 0},
 {'C': 289, 'A': 255, 'G': 313, 'T': 258, 'N': 1, 'S': 0}]

In [17]:
n[:5]

[{'C': 1160, 'A': 1007, 'G': 1361, 'T': 1143, 'N': 1, 'S': 0},
 {'C': 1135, 'A': 1052, 'G': 1293, 'T': 1192, 'N': 0, 'S': 0},
 {'C': 1185, 'A': 1039, 'G': 1270, 'T': 1178, 'N': 0, 'S': 0},
 {'C': 1182, 'A': 1037, 'G': 1255, 'T': 1197, 'N': 1, 'S': 0},
 {'C': 1164, 'A': 1046, 'G': 1276, 'T': 1185, 'N': 1, 'S': 0}]

In [18]:
pAn[:5]

[{'C': 1414, 'A': 1269, 'G': 1669, 'T': 1433, 'N': 3, 'S': 0},
 {'C': 1401, 'A': 1317, 'G': 1612, 'T': 1455, 'N': 3, 'S': 0},
 {'C': 1472, 'A': 1294, 'G': 1562, 'T': 1459, 'N': 1, 'S': 0},
 {'C': 1432, 'A': 1297, 'G': 1592, 'T': 1465, 'N': 2, 'S': 0},
 {'C': 1453, 'A': 1301, 'G': 1589, 'T': 1443, 'N': 2, 'S': 0}]

In [19]:
def id3(classes, attributes, sequences):
    """
    c- list of classes
    r- list of atributes
    s- list of objects
    """
    if not sequences:
        return -1
    if classes.count(classes[0]) == len(x):
        return classes[0]
    if not attributes:
        stats = {label: list(classes).count(label)  for label in np.unique(classes)}
        max(stats.items(), key=operator.itemgetter(1))[0]
    #atrybut maksymalizujący InfGain(D, S)
    d = 0

## atrybut maksimalizujący infGain

### entropia zbioru

In [20]:
testD = {'N': 1, 'T': 1143, 'S': 0, 'A': 1007, 'C': 1160, 'G': 1361}
for i in testD:
    print(i)

N
T
S
A
C
G


In [21]:
def calculate_frequency(pAn):
    """
    p = list of dictionaries caunting positive
    n - list of dictionaries counting negative
    y - output (0, 1)
    """
    totalInRow = [sum(row.values()) for row in pAn]
    frequencies = deepcopy(pAn)
    for row, rowSum in zip(frequencies, totalInRow):
        for key in row:
            row[key] = row[key]/rowSum
    return frequencies
frequencies = calculate_frequency(pAn)

In [22]:
def calculate_entropyLabel(labels):
    p = 0
    n = 0
    for ridx, row in enumerate(labels):
        if row == 1:
            p += 1
        else:
            n += 1
    pf = p/len(labels)
    nf = n/len(labels)
    entropy = -pf*log(pf, 2)-nf*log(nf, 2)
    return entropy

In [23]:
entropyL = calculate_entropyLabel(classes)

policzyc entropy dla outlook z przykładu zeby sprawdzic dzialanie

In [24]:
def singleEntropy(f, nrF):
    return -f*log(f, nrF)
def entropy(e1, e2, *rest):
    args = np.concatenate(([e1, e2], rest)).astype(float) 
    fs = [arg/sum(args) for arg in args]
    return sum([singleEntropy(f, len(fs)) for f in fs])
    

In [25]:
entropy(3, 3)

1.0

In [30]:
def informationGain(class1, class2, e1pair, e2pair, *rest):
    es = entropy(class1, class2)
#     eArgsP = np.concatenate(([e1pair, e2pair], rest)).astype(float)
    eArgsP = np.array([e1pair, e2pair]).astype(float)
    eArgs = [sum(e) for e in eArgsP]
    fs = [arg/sum(eArgs) for arg in eArgs]
    fsAtributes = [[value/sum(attribute) for value in attribute] for attribute in eArgsP]
    entropies = [entropy(value[0], value[1]) for value in fsAtributes]
    print(fs)
    print(fsAtributes)
    print(entropies)
    return sum([entropy*f for f, entropy in zip(fs, entropies)])
#     eSA = [[for ] for fs, entropy in zip(fsAtributes, entropies)]
#     return es-sum(eSA)
#     [f*entropy() for f in zip(fs, )]
#     eSA = sum()


In [31]:
informationGain(9, 5, (6, 2), (3, 3))

[0.5714285714285714, 0.42857142857142855]
[[0.75, 0.25], [0.5, 0.5]]
[0.8112781244591328, 1.0]


In [ ]:
informationGain(0.94, 8,  6, eW, eS)

In [ ]:
lista = [1, 2, 3, 4]
[arg/ for idx, arg in enumerate(lista)]